$\DeclareMathOperator{\opdiv}{div}$
Hybridization Techniques
===

Discretizing an elliptic equation by a primal method leads to a positive definite matrix. Using a mixed method leads to a saddle point problem. This disadvantage can be overcome by the so called hybridization technique:

The idea is as follows: Break the normal-continuity of the RT - space, and reinforce it by another Lagrange parameter. This new Lagrange parameter is a polynomial on every mesh edge (or face, in 3D). 

The variational formulation is as follows:

Find: $\sigma_h \in RT_k^{dc}, u_h \in P^k, \widehat u_h \in P^k(\cup E)$ such that

$$
\begin{array}{ccccccl}
\int_\Omega \sigma_h \tau_h & + & \sum_T \int_T \opdiv \tau_h \, u_h & + & \sum_E \int_E [\tau_{h,n}] \widehat u_h & = & 0 & \forall \, \tau_h \\
\sum_T \int_T \opdiv \sigma_h \, v_h & & & & & = & \int f v_h & \forall \, v_h \\
\sum_E \int_E [\sigma_{h,n}] \widehat v_h & & & & & = & 0 & \forall \,  \widehat v_h
\end{array}
$$

This formulation gives the same solution as the mixed formulation, so we don't need an extra error analysis

The physical meaning of the Lagrange paramter $\widehat u_h$ is the primal variable, what can be seen by integration by parts of the first  equation.

Now, Dirichlet boundary conditions are set by constraining the $\widehat u$ on the Dirichlet boundary, and Neumann boundary conditions are formulated by $\int_{\Gamma_N} g \widehat v$. The hybridized formulation is thus similar to a primal method.

The discretization system has the form

$$
\left( \begin{array}{ccc}
A & B_1^T & B_2^T \\
B_1 & & \\
B_2 & & 
\end{array} \right)
\left( \begin{array}{c}
\sigma \\ u_1 \\ u_2 
\end{array} \right)
=
\left( \begin{array}{c}
0 \\ f \\ 0
\end{array} \right)
$$

The submatrix 

$$
\left( \begin{array}{cc}
A & B_1^T \\
B_1 & 0
\end{array} \right)
$$

is regular and block-diagonal, each block corresponds to an element. Thus it can be cheaply inverted.

If $u_2$ were known, we could compute the first to variables from $u_2$:

$$
\left( \begin{array}{c}
\sigma \\ u_1
\end{array} \right)
= 
\left( \begin{array}{cc}
A & B_1^T \\
B_1
\end{array} \right)^{-1} 
\left[
\left( \begin{array}{c} 0 \\ f \end{array} \right)
- \left( \begin{array}{c} B_2^T \\ 0 \end{array} \right) u_2 
\right]
$$

Plug this term into the third equation $B_2 \sigma = 0$ we obtain the system

$$
\left( B_2 \; \; 0 \right)
\left( \begin{array}{cc}
A & B_1^T \\
B_1
\end{array} \right)^{-1} 
\left( \begin{array}{c} B_2^T \\ 0 \end{array} \right)
\; \; u_2 = 
\left( B_2 \; \; 0 \right)
\left( \begin{array}{cc}
A & B_1^T \\
B_1
\end{array} \right)^{-1} 
\left( \begin{array}{c} 0 \\ f \end{array} \right)
$$


The matrix on the left hand side is symmetric positive definite. It behaves like a standard system matrix (e.g. for condition number), and  standard iterative methods and preconditioners can be used for solution.


The lowest order hybrid method has the same degrees of freedom as the non-conforming $P^1$ element. Here, the extension to higher order is straight forward.

In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [2]:
order=2
Sigma = HDiv(mesh, order=order, discontinuous=True)
V = L2(mesh, order=order-1)
Vhat = FacetFESpace(mesh, order=order, dirichlet="left|bottom")
X = FESpace([Sigma, V, Vhat])

sigma,u,uhat = X.TrialFunction()
tau,v,vhat = X.TestFunction()

n = specialcf.normal(mesh.dim)

a = BilinearForm(X, eliminate_internal = True)
a += SymbolicBFI(-sigma*tau + div(sigma)*v + div(tau)*u)
a += SymbolicBFI(sigma*n*vhat+tau*n*uhat, element_boundary=True)

c = Preconditioner(a, "bddc")
a.Assemble()

f = LinearForm(X)
f += SymbolicLFI(v)
f.Assemble()

gfu = GridFunction(X)


f.vec.data += a.harmonic_extension_trans * f.vec

# gfu.vec.data = a.mat.Inverse(X.FreeDofs(True)) * f.vec
from ngsolve.solvers import CG
CG (mat=a.mat, pre=c.mat, rhs=f.vec, sol=gfu.vec, 
    printrates=True, maxsteps=200)

gfu.vec.data += a.harmonic_extension * gfu.vec
gfu.vec.data += a.inner_solve * f.vec

Draw (gfu.components[1])

it =  0  err =  0.37521481473746904
it =  1  err =  0.04262992433513675
it =  2  err =  0.007292126416452082
it =  3  err =  0.0016501410157702188
it =  4  err =  0.0004686524226871979
it =  5  err =  0.00016578310787995193
it =  6  err =  7.658530863629899e-05
it =  7  err =  2.4988863888473932e-05
it =  8  err =  8.822725447060745e-06
it =  9  err =  2.661214045774396e-06
it =  10  err =  8.997419154830874e-07
it =  11  err =  3.065745098050553e-07
it =  12  err =  9.543670575277324e-08
it =  13  err =  3.792768905458705e-08
it =  14  err =  1.0867518686980138e-08
it =  15  err =  4.127496004839697e-09
it =  16  err =  1.3167110462083571e-09
it =  17  err =  4.574751573006544e-10
it =  18  err =  1.4805248613335767e-10
it =  19  err =  4.715565797945443e-11
it =  20  err =  1.8764085011209077e-11
it =  21  err =  5.4601404903805035e-12
it =  22  err =  2.18531797342622e-12
it =  23  err =  5.901824289949845e-13
it =  24  err =  2.0396743338571256e-13
it =  25  err =  6.41990759785559